# CDC estimated inpatient beds time series

* [Estimated inpatient beds](https://healthdata.gov/sites/default/files/estimated_inpatient_all_20210221_1631.csv)
* [Estimated inpatient beds in use by COVID-19 patients](https://healthdata.gov/sites/default/files/estimated_inpatient_covid_20210221_1631.csv)
* [Estimated ICU beds](https://healthdata.gov/sites/default/files/estimated_icu_20210221_1631.csv)

In [1]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re
from bs4 import BeautifulSoup
import requests

In [2]:
# papermill parameters
output_folder = "../output/"

In [3]:
root_url = "https://healthdata.gov/dataset/covid-19-estimated-patient-impact-and-hospital-capacity-state"
states = [subdivision.code.replace("US-", \"\") for subdivision in pycountry.subdivisions if subdivision.code.startswith(\"US-\")]"
response = requests.get(root_url)
assert response.status_code == 200
soup = BeautifulSoup(response.content)
links = soup.find_all("a", {"class": "data-link"})
assert len(links)
df_list = []
for l in links:
    url = l["href"]
    print(url)
    df = pd.read_csv(url)
    df["ISO3166_1"] = "US"
    df["ISO3166_2"] = df["state"]
    
    df["Last_Reported_Flag"] = df["collection_date"].max() == df["collection_date"]
    df = df[df["ISO3166_2"].isin(states)]
    df_list.append(df)

https://healthdata.gov/sites/default/files/estimated_inpatient_all_20210223_1725.csv


NameError: name 'states' is not defined

In [ ]:
modified_date = soup.find("div", {"property": "dcterms:modified"}).text
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]

In [ ]:
inpatient_beds, covid_19_inpatient_beds, occupied_icu_beds, *_ = df_list
inpatient_beds.columns = ["STATE", "DATE", "INPATIENT_BEDS_OCCUPIED", "INPATIENT_BEDS_LOWER_BOUND", "INPATIENT_BEDS_UPPER_BOUND", "INPATIENT_BEDS_IN_USE_PCT", "INPATIENT_BEDS_IN_USE_PCT_LOWER_BOUND", "INPATIENT_BEDS_IN_USE_PCT_UPPER_BOUND", "TOTAL_INPATIENT_BEDS", "TOTAL_INPATIENT_BEDS_LOWER_BOUND", "TOTAL_INPATIENT_BEDS_UPPER_BOUND", "ISO3166_1", "ISO3166_2", "LAST_REPORTED_FLAG"]
covid_19_inpatient_beds.columns = inpatient_beds.columns
occupied_icu_beds.columns = ["STATE", "DATE", "STAFFED_ADULT_ICU_BEDS_OCCUPIED", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_LOWER_BOUND", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_UPPER_BOUND", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT_LOWER_BOUND", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT_UPPER_BOUND", "TOTAL_STAFFED_ICU_BEDS", "TOTAL_STAFFED_ICU_BEDS_LOWER_BOUND", "TOTAL_STAFFED_ICU_BEDS_UPPER_BOUND", "ISO3166_1", "ISO3166_2", "LAST_REPORTED_FLAG"]

```sql
CREATE TABLE CDC_INPATIENT_BEDS_ALL (
    STATE varchar,
DATE timestamp_ntz,
INPATIENT_BEDS_OCCUPIED integer,
INPATIENT_BEDS_LOWER_BOUND integer,
INPATIENT_BEDS_UPPER_BOUND integer,
INPATIENT_BEDS_IN_USE_PCT float,
INPATIENT_BEDS_IN_USE_PCT_LOWER_BOUND float,
INPATIENT_BEDS_IN_USE_PCT_UPPER_BOUND float,
TOTAL_INPATIENT_BEDS integer,
TOTAL_INPATIENT_BEDS_LOWER_BOUND integer,
TOTAL_INPATIENT_BEDS_UPPER_BOUND integer,
ISO3166_1 varchar(2),
ISO3166_2 varchar(2),
LAST_REPORTED_FLAG boolean
);



CREATE TABLE  CDC_INPATIENT_BEDS_COVID_19 (
  STATE varchar,
  DATE timestamp_ntz,
  INPATIENT_BEDS_OCCUPIED integer,
  INPATIENT_BEDS_LOWER_BOUND integer,
  INPATIENT_BEDS_UPPER_BOUND integer,
  INPATIENT_BEDS_IN_USE_PCT float,
  INPATIENT_BEDS_IN_USE_PCT_LOWER_BOUND float,
  INPATIENT_BEDS_IN_USE_PCT_UPPER_BOUND float,
  TOTAL_INPATIENT_BEDS integer,
  TOTAL_INPATIENT_BEDS_LOWER_BOUND integer,
  TOTAL_INPATIENT_BEDS_UPPER_BOUND integer,
  ISO3166_1 varchar(2),
  ISO3166_2 varchar(2),
  LAST_REPORTED_FLAG boolean
);


CREATE TABLE CDC_INPATIENT_BEDS_ICU_ALL(
  STATE varchar,
  DATE timestamp_ntz,
  STAFFED_ADULT_ICU_BEDS_OCCUPIED integer,
  STAFFED_ADULT_ICU_BEDS_OCCUPIED_LOWER_BOUND integer,
  STAFFED_ADULT_ICU_BEDS_OCCUPIED_UPPER_BOUND integer,
  STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT float,
  STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT_LOWER_BOUND float,
  STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT_UPPER_BOUND float,
  TOTAL_STAFFED_ICU_BEDS integer,
  TOTAL_STAFFED_ICU_BEDS_LOWER_BOUND integer,
  TOTAL_STAFFED_ICU_BEDS_UPPER_BOUND integer,
  ISO3166_1 varchar(2),
  ISO3166_2 varchar(2),
  LAST_REPORTED_FLAG boolean
);
```

In [ ]:
inpatient_beds.to_csv(output_folder + "CDC_INPATIENT_BEDS_ALL.csv", index=False, columns=inpatient_beds.columns)
covid_19_inpatient_beds.to_csv(output_folder + "CDC_INPATIENT_BEDS_COVID_19.csv", index=False, columns=covid_19_inpatient_beds.columns)
occupied_icu_beds.to_csv(output_folder + "CDC_INPATIENT_BEDS_ICU_ALL.csv", index=False, columns=occupied_icu_beds.columns)